In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from collections import defaultdict, Counter
import os
from datetime import timedelta
import plotly.express as px


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [81]:
weather_data = pd.read_csv('cleaned_weather.csv')

data = pd.read_csv('more_resource.csv')

In [82]:
weather_data.head()

,Unnamed: 0,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,datetime,year,month,day,wind_direction_cos,wind_direction_sin,TS,GR,RA,DZ,SN,FG,BR,HZ,FU,SQ,MI,BC,VC,FG+,MD
0,0,1,83,50,67.0,14.0,51,56.0,0.0,2.0,448,1849,0.0,29.10,29.82,1.7,270,9.2,2007-05-01,2007,5,1,-1.836970e-16,-1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,2,84,52,68.0,14.0,51,57.0,0.0,3.0,448,1849,0.0,29.18,29.82,2.7,250,9.6,2007-05-01,2007,5,1,-3.420201e-01,-0.939693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,1,59,42,51.0,-3.0,42,47.0,14.0,0.0,447,1850,0.0,29.38,30.09,13.0,40,13.4,2007-05-02,2007,5,2,7.660444e-01,0.642788,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,3,2,60,43,52.0,-3.0,42,47.0,13.0,0.0,447,1850,0.0,29.44,30.08,13.3,20,13.4,2007-05-02,2007,5,2,9.396926e-01,0.342020,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4,4,1,66,46,56.0,2.0,40,48.0,9.0,0.0,446,1851,0.0,29.39,30.12,11.7,70,11.9,2007-05-03,2007,5,3,3.420201e-01,0.939693,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [83]:
weather_data.drop(columns = ['Unnamed: 0'], inplace=True)
weather_data.head()

,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,datetime,year,month,day,wind_direction_cos,wind_direction_sin,TS,GR,RA,DZ,SN,FG,BR,HZ,FU,SQ,MI,BC,VC,FG+,MD
0,1,83,50,67.0,14.0,51,56.0,0.0,2.0,448,1849,0.0,29.10,29.82,1.7,270,9.2,2007-05-01,2007,5,1,-1.836970e-16,-1.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,84,52,68.0,14.0,51,57.0,0.0,3.0,448,1849,0.0,29.18,29.82,2.7,250,9.6,2007-05-01,2007,5,1,-3.420201e-01,-0.939693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,59,42,51.0,-3.0,42,47.0,14.0,0.0,447,1850,0.0,29.38,30.09,13.0,40,13.4,2007-05-02,2007,5,2,7.660444e-01,0.642788,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,2,60,43,52.0,-3.0,42,47.0,13.0,0.0,447,1850,0.0,29.44,30.08,13.3,20,13.4,2007-05-02,2007,5,2,9.396926e-01,0.342020,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0
4,1,66,46,56.0,2.0,40,48.0,9.0,0.0,446,1851,0.0,29.39,30.12,11.7,70,11.9,2007-05-03,2007,5,3,3.420201e-01,0.939693,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [84]:
weather_data.dtypes

Station                   int64
Tmax                      int64
Tmin                      int64
Tavg                    float64
Depart                  float64
DewPoint                  int64
WetBulb                 float64
Heat                    float64
Cool                    float64
Sunrise                   int64
Sunset                    int64
water_equiv_snowfall    float64
station_pressure        float64
sealevel_pressure       float64
wind_speed              float64
wind_direction            int64
avg_wind_speed          float64
datetime                 object
year                      int64
month                     int64
day                       int64
wind_direction_cos      float64
wind_direction_sin      float64
TS                        int64
GR                        int64
RA                        int64
DZ                        int64
SN                        int64
FG                        int64
BR                        int64
HZ                        int64
FU      

In [85]:
ohare, midway = dict(), dict()

for lag in range(1,4):
    delta = timedelta(days=lag)
    x = weather_data.loc[weather_data.Station==1, :].copy()
    x['datetime'] = pd.to_datetime(x['datetime']) + delta
    x.drop(columns=['year', 'month', 'day', 'Station', 'Sunrise', 'Sunset'], inplace=True)
    x['datetime']=x['datetime'].dt.strftime('%Y-%m-%d')
    ohare[lag] = x
    y = weather_data.loc[weather_data.Station==2, :].copy()
    y['datetime'] = pd.to_datetime(y['datetime']) + delta
    y.drop(columns=['year', 'month', 'day', 'Station', 'Sunrise', 'Sunset'], inplace=True)
    y['datetime'] = y['datetime'].dt.strftime('%Y-%m-%d')
    midway[lag] = y

In [86]:
data.drop(columns = ['Unnamed: 0'], inplace=True)
data.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,datetime,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,lat,lon,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.040310,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.040310,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.163721,-0.000009,0.163721,1.414,0,1.0,1.0,29043,72086.0,5741.46
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.110190,-0.020911,0.108188,1.414,0,1.0,1.0,28982,55831.0,7590.96
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,T015,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.110190,-0.020911,0.108188,1.414,0,1.0,1.0,28982,55831.0,7590.96


In [87]:
def ohare_dist(data):
    x, y = 41.995, -87.933
    return np.sqrt((data['Latitude']-x)**2+(data['Longitude']-y)**2)

def midway_dist(data):
    x, y = 41.786, -87.752
    return np.sqrt((data['Latitude']-x)**2+(data['Longitude']-y)**2)


def weather_lag_feature(data, ohare, midway):
    o_dist = ohare_dist(data)
    m_dist = midway_dist(data)
    all_col = set(col for col in data.columns)
    for lag in ohare:
        o_weather, m_weather = ohare[lag], midway[lag]
        col = [c for c in o_weather.columns if c in all_col]
        temp_o = data.merge(o_weather[col], on=['datetime'], how='left', suffixes=(None, '_'+str(lag)))
        temp_m = data.merge(m_weather[col], on=['datetime'], how='left', suffixes=(None, '_'+str(lag)))
        mask = (m_dist.values<o_dist.values)
        new_col = [c+'_'+str(lag) for c in col if c!='datetime']
        temp_o.loc[mask, new_col] = temp_m.loc[mask, new_col]
        data = temp_o
    return data    
        

In [88]:
data = weather_lag_feature(data, ohare, midway)

In [89]:
data.head(3)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,datetime,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,lat,lon,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.040310,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.040310,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.163721,-0.000009,0.163721,1.414,0,1.0,1.0,29043,72086.0,5741.46,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0


In [90]:
data.isna().sum().sort_values(ascending=False)

Date                      0
Address                   0
wind_direction_cos_2      0
avg_wind_speed_2          0
wind_direction_2          0
wind_speed_2              0
sealevel_pressure_2       0
station_pressure_2        0
water_equiv_snowfall_2    0
Cool_2                    0
Heat_2                    0
WetBulb_2                 0
DewPoint_2                0
Depart_2                  0
Tavg_2                    0
Tmin_2                    0
Tmax_2                    0
MD_1                      0
VC_1                      0
HZ_1                      0
BR_1                      0
FG_1                      0
DZ_1                      0
RA_1                      0
TS_1                      0
wind_direction_sin_1      0
wind_direction_cos_1      0
avg_wind_speed_1          0
wind_direction_1          0
wind_speed_1              0
sealevel_pressure_1       0
wind_direction_sin_2      0
TS_2                      0
RA_2                      0
station_pressure_3        0
VC_3                

In [91]:
x = data[['AddressNumberAndStreet', 'NumMosquitos', 
          'WnvPresent', 'datetime']].groupby(['datetime']).agg({'AddressNumberAndStreet':np.size, 'NumMosquitos':np.sum, 'WnvPresent':np.sum})
x['density'] = x['WnvPresent']/x['NumMosquitos']
x

,AddressNumberAndStreet,NumMosquitos,WnvPresent,density
datetime,,,,
2007-05-29,25,40,0,0.000000
2007-06-05,60,127,0,0.000000
2007-06-26,70,216,0,0.000000
2007-06-29,46,85,0,0.000000
2007-07-02,74,240,0,0.000000
2007-07-11,152,1103,0,0.000000
2007-07-18,129,853,1,0.001172
2007-07-19,45,687,0,0.000000
2007-07-25,92,4069,5,0.001229


In [92]:
def density_lag(data):
    all_dates = list(np.sort(data['datetime'].unique()))
    def helper(x, l=all_dates):
        idx = l.index(x)
        if idx<len(l)-1:
            return l[idx+1]
        return ''
    all_location = data['AddressNumberAndStreet'].unique()
    density = data[['AddressNumberAndStreet', 
          'datetime', 'NumMosquitos', 'WnvPresent']].groupby(['AddressNumberAndStreet', 'datetime']).agg({'NumMosquitos':np.sum, 'WnvPresent':np.sum}).reset_index()
    
    density['density'] = density['WnvPresent']/density['NumMosquitos']
    density['datetime'] = density['datetime'].apply(lambda x:helper(x))
    data = data.merge(density[['AddressNumberAndStreet', 'datetime', 'density']].copy(), on=['AddressNumberAndStreet', 'datetime'], how='left')
    data = data.merge(density[['AddressNumberAndStreet', 'datetime', 'NumMosquitos']].copy(), on=['AddressNumberAndStreet', 'datetime'], how='left', suffixes=(None, '_lag'))
    mask = data['density'].isna()
    density_map = density[['AddressNumberAndStreet', 'density']].groupby('AddressNumberAndStreet').agg(np.median).reset_index()
    density_map = pd.Series(data=density_map['density'].values, index = density_map['AddressNumberAndStreet'].values)
    data['density'][mask] = data['AddressNumberAndStreet'].map(density_map)[mask]
    count_map = density[['AddressNumberAndStreet', 'NumMosquitos']].groupby('AddressNumberAndStreet').agg(np.median).reset_index()
    count_map = pd.Series(data=count_map['NumMosquitos'].values, index = count_map['AddressNumberAndStreet'].values)
    data['NumMosquitos_lag'][mask] = data['AddressNumberAndStreet'].map(count_map)[mask]
    return data
    
    

In [93]:
data = density_lag(data)
data.isna().sum().sort_values(ascending=False)

C:\Users\lmaow\AppData\Local\Temp\ipykernel_17920\3565492560.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['density'][mask] = data['AddressNumberAndStreet'].map(density_map)[mask]
C:\Users\lmaow\AppData\Local\Temp\ipykernel_17920\3565492560.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['NumMosquitos_lag'][mask] = data['AddressNumberAndStreet'].map(count_map)[mask]


Date                      0
RA_2                      0
wind_direction_sin_2      0
wind_direction_cos_2      0
avg_wind_speed_2          0
wind_direction_2          0
wind_speed_2              0
sealevel_pressure_2       0
station_pressure_2        0
water_equiv_snowfall_2    0
Cool_2                    0
Heat_2                    0
WetBulb_2                 0
DewPoint_2                0
Depart_2                  0
Tavg_2                    0
Tmin_2                    0
Tmax_2                    0
MD_1                      0
VC_1                      0
HZ_1                      0
BR_1                      0
FG_1                      0
DZ_1                      0
RA_1                      0
TS_1                      0
wind_direction_sin_1      0
wind_direction_cos_1      0
avg_wind_speed_1          0
wind_direction_1          0
wind_speed_1              0
TS_2                      0
DZ_2                      0
Address                   0
FG_2                      0
density             

In [94]:
data.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,datetime,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,lat,lon,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,2007-05-29,5,88,60,74.0,10.0,58,65.0,0.0,9.0,421,1917,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,41.995,-87.933,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0


In [ ]:
data['daytime'] = data['Sunset']-data['Sunrise']

In [95]:
drop_cols = ['Date', 'Address', 'Trap', 'AddressNumberAndStreet', 
             'Latitude', 'Longitude', 'AddressAccuracy', 'datetime', 'lat', 'lon', 'Sunrise', 'Sunset']
data.drop(columns=drop_cols, inplace=True)
data.head(2)

,Species,Block,Street,NumMosquitos,WnvPresent,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,1,0,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0
1,CULEX RESTUANS,41,N OAK PARK AVE,1,0,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0


In [96]:
def IV(data, feature, y):
    temp = data[feature].copy()
    all_values = data[feature].unique()
    if len(all_values)>20 and temp.dtype!='object':
        temp = pd.qcut(temp, q=20, labels=False, duplicates='drop')
        all_values = temp.unique()
    res = 0
    for v in all_values:
        mask = (temp.values==v)
        n = (mask).sum()
        res+=((n-y[mask].sum())/n-(y[mask].sum())/n)*np.log((n-y[mask].sum())/(y[mask].sum()+0.01))
    return res    

In [97]:
iv_score = defaultdict(float)
for feature in data.select_dtypes(exclude=['object']).columns:
    if feature!='WnvPresent':
        iv_score[feature] = IV(data, feature, data['WnvPresent'].values)
    
iv_score = pd.Series(data=list(iv_score.values()), index=list(iv_score.keys()))    

Display information value in descending order.

In [98]:
iv_score.sort_values(ascending=False)

Heat                      94.709436
Heat_1                    72.245112
station_pressure_2        66.009071
Tmax_2                    63.493943
DewPoint                  62.753229
sealevel_pressure_1       62.494700
Heat_2                    61.450461
Heat_3                    61.396908
WetBulb                   61.344391
Depart_1                  61.264618
wind_speed_1              60.329050
wind_direction_cos_1      59.062885
wind_direction_sin_2      58.414710
station_pressure_1        56.863805
avg_wind_speed            56.846169
sealevel_pressure_2       56.823941
wind_speed_2              56.532765
station_pressure_3        55.949869
wind_direction_2          55.809218
avg_wind_speed_2          55.649097
Tmax_3                    55.345176
wind_speed_3              55.340893
population_density        54.874135
wind_direction_sin_1      54.658073
sealevel_pressure_3       54.616891
wind_speed                53.982387
station_pressure          53.729761
avg_wind_speed_1          53

In [20]:
iv_score = defaultdict(float)
for feature in data.select_dtypes(include=['object']).columns:
    if feature!='WnvPresent':
        iv_score[feature] = IV(data, feature, data['WnvPresent'].values)
    
iv_score = pd.Series(data=list(iv_score.values()), index=list(iv_score.keys()))    

In [21]:
iv_score.sort_values(ascending=False)

Street     516.526617
Species     38.372784
dtype: float64

In [22]:
from sklearn.preprocessing import OrdinalEncoder

for feature in data.select_dtypes(include=['object']).columns:
    encoder = OrdinalEncoder(dtype=np.int64)
    data[feature] = encoder.fit_transform(data[[feature]].values)[:, 0]
    
data.head(2)    

,Species,Block,Street,NumMosquitos,WnvPresent,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,2,41,31,1,0,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0
1,3,41,31,1,0,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0


In [58]:
y = data['WnvPresent']
X = data.drop(columns=['WnvPresent'])

In [60]:
X.head(2)

,Species,Block,Street,NumMosquitos,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,2,41,31,1,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0
1,3,41,31,1,5,88,60,74.0,10.0,58,65.0,0.0,9.0,0.0,29.39,30.11,5.8,180,6.5,-1.0,1.224647e-16,0,0,0,0,1,1,0,0,1.414,0,1.0,1.0,0.138026,-0.04031,0.132009,1.414,0,1.0,1.0,75604,55324.0,10611.96,79,50,65.0,2.0,47,56.0,0.0,0.0,0.0,29.43,30.15,4.8,100,5.3,-0.173648,0.984808,0,1,0,0,0,0,0,0,75,57,66.0,4.0,47,56.0,0.0,1.0,0.04,29.34,30.06,8.9,260,9.5,-0.173648,-0.984808,0,1,0,0,1,0,0,0,65,49,57.0,-5.0,55,56.0,8.0,0.0,1.01,29.36,30.11,2.8,170,7.2,-0.984808,0.173648,1,1,0,0,1,1,0,0,0.0,27.0


In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance

baseline =  RandomForestClassifier(max_depth=32, min_samples_leaf=2, min_samples_split=4, class_weight={0:1, 1:20})

baseline.fit(X, y)

result = permutation_importance(baseline, X, y, n_repeats=50, scoring = 'balanced_accuracy', random_state=41)

In [69]:
importance = pd.DataFrame(result.importances.T)
importance.rename(columns = {i:x for i, x in enumerate(list(X.columns))}, inplace=True)
importance.head(2)

,Species,Block,Street,NumMosquitos,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,0.115905,0.045590,0.051386,0.300852,0.134683,0.002575,0.004591,0.009838,0.005197,0.006409,0.015132,0.000707,0.018303,0.003278,0.007313,0.009228,0.010792,0.01034,0.012101,0.003379,0.002575,-0.0001,-0.000151,-0.000308,0.00005,-0.000201,0.000857,-0.0001,0.002371,0.004792,-0.000151,0.001463,0.014472,0.046946,0.044786,0.048459,0.000807,0.0,0.000707,-0.000151,0.039640,0.042462,0.040694,0.004239,0.010849,0.002622,0.002977,0.028446,0.027538,0.0,0.005093,0.002776,0.007367,0.004845,0.004594,0.003784,0.007517,0.004182,0.010491,0.001711,0.000606,0.000907,0.0,-0.000452,-0.000100,-0.00005,-0.0001,0.004139,0.003027,0.004942,0.002927,0.003328,0.008579,0.00000,0.00227,0.003077,0.003184,0.007976,0.006613,0.002069,0.004038,0.009386,0.005297,-0.000201,0.001664,0.0,0.000000,0.001008,-0.00005,0.0,0.000703,0.002019,0.003027,0.002876,0.003131,0.000405,0.004993,0.00000,0.003178,0.001212,0.005656,0.014181,0.003884,0.002927,0.003231,0.002927,0.003834,-0.0001,0.001765,0.0,0.0,0.000757,-0.000100,0.0,0.000656,0.012302,0.099742
1,0.113634,0.045134,0.046594,0.273934,0.121427,0.008927,0.005699,0.018762,0.006155,0.009583,0.015383,0.000000,0.020925,0.005244,0.011094,0.010893,0.006861,0.01019,0.012607,0.003228,0.008120,-0.0001,0.000707,0.000650,0.00005,-0.000201,0.000807,0.0000,0.000656,0.007514,-0.000100,0.002371,0.017951,0.045335,0.042817,0.051433,0.000807,0.0,0.000757,-0.000151,0.039184,0.044729,0.041347,0.005247,0.012008,0.002270,0.005699,0.026731,0.020580,0.0,0.004236,0.001718,0.007367,0.008174,0.002324,0.008522,0.007467,0.006754,0.011499,0.002418,0.000807,0.000907,0.0,0.000355,0.000606,0.00000,0.0000,0.003382,0.003178,0.004691,0.002019,0.002170,0.006563,-0.00005,0.00227,-0.000402,0.003586,0.011050,0.003941,0.003633,0.004745,0.007772,0.005753,-0.000050,0.001614,0.0,0.000907,0.000807,-0.00005,0.0,0.000804,0.002019,0.004440,0.003583,0.004343,0.004792,0.002069,0.00005,0.002270,0.000355,0.003184,0.024263,0.002726,0.003834,0.002374,0.001011,0.003128,-0.0001,0.000807,0.0,0.0,-0.000100,-0.000251,0.0,-0.000151,0.011043,0.094903


Permutation test score using random forest model.

In [70]:
importance.mean(axis=0).sort_values(ascending=False)

NumMosquitos              0.288306
month                     0.131129
Species                   0.113926
NumMosquitos_lag          0.097881
station_lon_diff          0.049997
station_lat_diff          0.048715
Block                     0.046980
Street                    0.046369
station_dist              0.046205
median_income             0.044402
population_density        0.043772
population                0.041264
DewPoint_1                0.028937
WetBulb_1                 0.028373
sealevel_pressure_3       0.021292
Cool                      0.020742
WetBulb                   0.019139
Tavg                      0.018779
spray_lon_diff            0.016433
Tmin_1                    0.014350
density                   0.012103
sealevel_pressure_2       0.010989
DewPoint                  0.010944
wind_direction_sin_1      0.010753
sealevel_pressure         0.010596
avg_wind_speed            0.010534
wind_direction_cos_2      0.009750
station_pressure          0.009735
sealevel_pressure_1 

In [72]:
(importance<=0).sum(axis=0).sort_values()

Species                    0
density                    0
wind_direction_cos_1       0
avg_wind_speed_1           0
wind_direction_1           0
wind_speed_1               0
sealevel_pressure_1        0
station_pressure_1         0
water_equiv_snowfall_1     0
Cool_1                     0
WetBulb_1                  0
DewPoint_1                 0
Depart_1                   0
Tmin_1                     0
Tmax_1                     0
population_density         0
Tmax_2                     0
Tmin_2                     0
Tavg_2                     0
DewPoint_2                 0
wind_direction_cos_3       0
wind_direction_3           0
wind_speed_3               0
sealevel_pressure_3        0
station_pressure_3         0
Cool_3                     0
WetBulb_3                  0
median_income              0
Tavg_3                     0
wind_direction_sin_2       0
wind_direction_cos_2       0
avg_wind_speed_2           0
wind_speed_2               0
sealevel_pressure_2        0
station_pressu

In [73]:
importance.std(axis=0).sort_values(ascending=False)

NumMosquitos              0.009611
month                     0.008115
Species                   0.007146
NumMosquitos_lag          0.007110
station_lat_diff          0.005396
population                0.004132
station_lon_diff          0.004127
Block                     0.004059
Tavg                      0.003936
median_income             0.003761
Street                    0.003695
WetBulb_1                 0.003579
WetBulb                   0.003538
sealevel_pressure_3       0.003536
Cool                      0.003484
station_dist              0.003398
population_density        0.003190
DewPoint_1                0.003172
Tmin_1                    0.003004
wind_direction_sin_1      0.002712
wind_direction_cos_2      0.002493
station_pressure_3        0.002439
wind_speed                0.002275
DewPoint                  0.002260
sealevel_pressure_1       0.002241
wind_direction_sin_2      0.002229
sealevel_pressure_2       0.002192
sealevel_pressure         0.002190
wind_speed_1        

In [74]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline

lg = LogisticRegression(C=4, class_weight={0:1,1:20})
power = PowerTransformer()

pipe = Pipeline([('p', power), ('lg', lg)])

pipe.fit(X, y)

result = permutation_importance(pipe, X, y, n_repeats=50, scoring = 'balanced_accuracy', random_state=41)

C:\folderconda\envs\new_env\lib\site-packages\sklearn\preprocessing\_data.py:3253: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())
C:\folderconda\envs\new_env\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [75]:
lg_importance = pd.DataFrame(result.importances.T)
lg_importance.rename(columns = {i:x for i, x in enumerate(list(X.columns))}, inplace=True)
lg_importance.head(2)

,Species,Block,Street,NumMosquitos,month,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,water_equiv_snowfall,station_pressure,sealevel_pressure,wind_speed,wind_direction,avg_wind_speed,wind_direction_cos,wind_direction_sin,TS,RA,DZ,FG,BR,HZ,VC,MD,spray_dist,has_spray,spray_lat_diff,spray_lon_diff,station_dist,station_lat_diff,station_lon_diff,pre_spray_dist,pre_has_spray,pre_spray_lat_diff,pre_spray_lon_diff,population,median_income,population_density,Tmax_1,Tmin_1,Tavg_1,Depart_1,DewPoint_1,WetBulb_1,Heat_1,Cool_1,water_equiv_snowfall_1,station_pressure_1,sealevel_pressure_1,wind_speed_1,wind_direction_1,avg_wind_speed_1,wind_direction_cos_1,wind_direction_sin_1,TS_1,RA_1,DZ_1,FG_1,BR_1,HZ_1,VC_1,MD_1,Tmax_2,Tmin_2,Tavg_2,Depart_2,DewPoint_2,WetBulb_2,Heat_2,Cool_2,water_equiv_snowfall_2,station_pressure_2,sealevel_pressure_2,wind_speed_2,wind_direction_2,avg_wind_speed_2,wind_direction_cos_2,wind_direction_sin_2,TS_2,RA_2,DZ_2,FG_2,BR_2,HZ_2,VC_2,MD_2,Tmax_3,Tmin_3,Tavg_3,Depart_3,DewPoint_3,WetBulb_3,Heat_3,Cool_3,water_equiv_snowfall_3,station_pressure_3,sealevel_pressure_3,wind_speed_3,wind_direction_3,avg_wind_speed_3,wind_direction_cos_3,wind_direction_sin_3,TS_3,RA_3,DZ_3,FG_3,BR_3,HZ_3,VC_3,MD_3,density,NumMosquitos_lag
0,0.002937,-0.000201,0.006305,0.124099,0.167119,0.029279,0.033056,0.003757,0.004139,0.086009,-0.000847,0.001021,0.163158,0.000603,0.040583,0.023630,0.004002,0.003419,0.000355,0.004293,0.010243,-0.004232,-0.006265,0.004835,0.001209,0.011197,0.001714,-0.000352,0.019153,0.000606,0.000606,0.000606,0.000606,0.023131,0.005502,-0.001915,-0.000151,-0.000151,-0.000151,-0.000151,0.005793,0.0,0.000556,0.044521,0.013274,-0.001855,0.044109,0.019538,0.033481,0.022428,0.089729,0.009419,0.036538,0.010856,0.006573,-0.000100,0.050977,0.075374,0.099420,-0.001664,0.005850,-0.001564,0.002665,0.00005,0.008167,-0.000201,0.015714,-0.002210,0.033177,0.051075,0.009476,0.008569,0.006258,-0.000502,0.024976,0.004661,-0.000556,0.000024,0.019522,0.002682,0.051921,0.002816,0.022047,0.006245,0.011726,0.001011,0.009265,0.004644,0.007755,0.00000,0.056810,0.059184,0.060936,0.003174,0.073147,0.004433,0.006412,0.009453,0.055193,0.027736,0.009908,0.012319,0.003975,0.003278,0.001273,0.005893,0.007055,0.008264,0.041739,0.015828,0.000301,0.009429,0.002367,0.000003,0.021464,-0.000703,-0.000355
1,0.001423,-0.000301,0.004440,0.123241,0.191018,0.046366,0.040302,0.007788,0.007316,0.080618,0.005100,0.007122,0.146369,0.001058,0.041544,0.035983,0.005056,0.003931,0.001162,0.006510,0.017600,0.004340,-0.002532,0.005843,0.000352,0.005803,-0.000151,0.000907,0.009727,0.000807,0.000807,0.000807,0.000807,0.024193,0.002478,-0.000804,0.000000,0.000000,0.000000,0.000000,-0.002069,0.0,0.002471,0.044210,0.018162,0.003084,0.043908,0.014850,0.026677,0.023791,0.081760,0.001457,0.047331,0.011157,0.013936,0.000807,0.039429,0.071935,0.108086,-0.003529,0.003278,0.001611,0.002160,0.00010,0.004189,-0.000050,0.015460,0.000211,0.037061,0.048657,0.022991,0.003020,0.012108,0.002521,0.023921,0.003496,-0.000506,0.005867,0.015741,0.002321,0.052675,0.012748,0.028998,0.007912,0.008200,-0.000653,0.005535,0.002374,0.010628,-0.00005,0.052973,0.063223,0.049541,-0.001356,0.081224,-0.000201,0.023148,0.010923,0.058568,0.036606,0.016565,0.013829,0.009319,0.003228,-0.003971,0.015925,0.012446,0.000000,0.034078,0.008924,0.001108,-0.000047,0.001510,0.008120,0.030290,-0.002468,-0.000154


Permutation test score for logistic regression model

In [76]:
lg_importance.mean(axis=0).sort_values(ascending=False)

month                     0.171665
Cool                      0.157968
NumMosquitos              0.128922
wind_direction_sin_1      0.105341
Cool_1                    0.093563
Depart_3                  0.087772
DewPoint                  0.086999
wind_direction_cos_1      0.066127
Tmax_3                    0.065988
MD_2                      0.062724
Tmin_3                    0.058233
avg_wind_speed_1          0.056858
Cool_3                    0.055215
Tavg_2                    0.049488
RA_3                      0.045281
avg_wind_speed_2          0.044949
station_pressure_1        0.041987
station_pressure          0.037303
Tmax_1                    0.036785
Depart_1                  0.036774
Tmax                      0.036668
sealevel_pressure         0.035059
Tmin                      0.032954
water_equiv_snowfall_3    0.031134
Tmin_2                    0.029144
wind_direction_sin_2      0.028470
station_dist              0.026785
MD_3                      0.026287
WetBulb_1           

In [77]:
(lg_importance>0).sum(axis=0).sort_values(ascending=False)

wind_direction_sin_1      50
Depart_1                  50
sealevel_pressure_1       50
station_pressure_1        50
Cool_1                    50
Heat_1                    50
WetBulb_1                 50
DewPoint_1                50
Tmax_3                    50
MD_2                      50
Tmin_1                    50
Tmax_1                    50
Tmin_3                    50
Depart_3                  50
Heat_3                    50
Cool_3                    50
wind_speed_1              50
avg_wind_speed_1          50
station_pressure_3        50
Depart_2                  50
wind_speed_2              50
sealevel_pressure_2       50
wind_direction_sin_2      50
water_equiv_snowfall_2    50
Cool_2                    50
WetBulb_2                 50
Tavg_2                    50
wind_direction_cos_1      50
Tmin_2                    50
MD_1                      50
RA_2                      50
FG_2                      50
RA_1                      50
HZ_2                      50
water_equiv_sn

In [78]:
lg_importance.std(axis=0).sort_values(ascending=False)

month                     0.010194
NumMosquitos              0.009783
Cool_1                    0.009041
Cool                      0.008684
wind_direction_sin_1      0.007966
wind_direction_cos_1      0.007962
Tmax_3                    0.007716
DewPoint                  0.007650
Depart_3                  0.007509
avg_wind_speed_1          0.007374
Cool_3                    0.007165
station_pressure_1        0.007032
Tmin_3                    0.007005
MD_2                      0.006886
Tmax_1                    0.006819
Depart_1                  0.006819
avg_wind_speed_2          0.006540
Tavg_2                    0.006510
Tmin_2                    0.006445
MD_3                      0.006269
RA_3                      0.006018
station_pressure          0.005972
sealevel_pressure         0.005923
Tmax                      0.005687
WetBulb_3                 0.005559
WetBulb_1                 0.005417
sealevel_pressure_3       0.005350
Tmin                      0.005318
Cool_2              